# 1 Processing

## Praperation

In [1]:
import os
HOME = os.getcwd()
print(HOME)

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

/home/jpan/xAI_Masterthesis_Pan/Code/Detection/YOLO/202407_Mixed_tests


In [2]:
#clearml mac
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=QO1VBBX9J2S2VYILQTGI
# %env CLEARML_API_SECRET_KEY=ERuc1S6o5SirQGugvYXDFjH9b9aNi0u8S3rpALzXMa8YPSLDMW

# clearml win
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
%env CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
env: CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn


In [3]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.41 🚀 Python-3.12.2 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
Setup complete ✅ (24 CPUs, 125.5 GB RAM, 321.3/459.6 GB disk)


/home/jpan/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Config

In [12]:
# read config file
import yaml

def load_config(config_file):
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    return config

config = load_config('Mixed_d1+d2+sf1_config_2.yaml')

print(config)

# project
project = config['project']
name = config['name']
exist_ok = config['exist_ok']

# dataset
data_folder = config['data_server']
data = config['data_yaml']
test_data = config['test_data']
imgsz = config['imgsz']

# model
model = config['model']
task = config['task']
mode = config['mode']
# device = config['device_mac']
device = config['device_win']

# Training parameters
epochs = config['epochs']
batch = config['batch']
lr0 = config['lr0']
cos_lr = config['cos_lr']
weight_decay = config['weight_decay']
dropout = config['dropout']
optimizer = config['optimizer']
momentum = config['momentum']
seed = config['seed']

# Output configuration
# save = config['save']
# save_period = config['save_period']
plots = config['plots']
test_output = config['test_output']


{'project': 'Uni-Wood', 'name': 'Mixed_YOLO_d1+d2+s1_test_2', 'exist_ok': False, 'data_server': '/mnt/data/backup_pan/Dataset/Detection/Mixed_datasets/Det-dominik_v1-v2-sf_v1.v2i.yolov8', 'data_yaml': '/mnt/data/backup_pan/Dataset/Detection/Mixed_datasets/Det-dominik_v1-v2-sf_v1.v2i.yolov8/data.yaml', 'test_data': '/path/to/test_data.txt', 'imgsz': 640, 'model': 'yolov8m.yaml', 'task': 'detect', 'mode': 'train', 'device_mac': 'mps', 'device_win': 0, 'epochs': 300, 'batch': 16, 'lr0': 0.001, 'cos_lr': True, 'weight_decay': 0.0005, 'dropout': 0.0, 'optimizer': 'Adam', 'momentum': 0.9, 'seed': 42, 'save': False, 'save_period': -1, 'plots': True, 'test_output': '/path/to/test_output'}


## Split data

In [5]:
import os
import shutil
import random

train_dir = os.path.join(data_folder, "train")
val_dir = os.path.join(data_folder, "valid")
test_dir = os.path.join(data_folder, "test")

os.makedirs(os.path.join(val_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "labels"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "labels"), exist_ok=True)

val_ratio = 0.1
test_ratio = 0.05

random.seed(seed)

image_files = os.listdir(os.path.join(train_dir, "images"))
label_files = os.listdir(os.path.join(train_dir, "labels"))

image_files.sort()
label_files.sort()

data_pairs = list(zip(image_files, label_files))

random.shuffle(data_pairs)

total_files = len(data_pairs)
val_count = int(total_files * val_ratio)
test_count = int(total_files * test_ratio)

val_data = data_pairs[:val_count]
test_data = data_pairs[val_count:val_count+test_count]
train_data = data_pairs[val_count+test_count:]



In [9]:
def move_files(data, src_dir, dest_dir):
    for img_file, lbl_file in data:
        shutil.move(os.path.join(src_dir, "images", img_file), os.path.join(dest_dir, "images", img_file))
        shutil.move(os.path.join(src_dir, "labels", lbl_file), os.path.join(dest_dir, "labels", lbl_file))

move_files(val_data, train_dir, val_dir)
move_files(test_data, train_dir, test_dir)

print("Split done!")

Split done!


In [10]:
def count_files(directory):
    images_count = len(os.listdir(os.path.join(directory, "images")))
    labels_count = len(os.listdir(os.path.join(directory, "labels")))
    return images_count, labels_count

train_images_count, train_labels_count = count_files(train_dir)
val_images_count, val_labels_count = count_files(val_dir)
test_images_count, test_labels_count = count_files(test_dir)

print(f"Train set: {train_images_count} images, {train_labels_count} labels")
print(f"Validation set: {val_images_count} images, {val_labels_count} labels")
print(f"Test set: {test_images_count} images, {test_labels_count} labels")

Train set: 4356 images, 4356 labels
Validation set: 424 images, 424 labels
Test set: 212 images, 212 labels


In [7]:
def move_back_files(src_dir, dest_dir):
    for file_name in os.listdir(os.path.join(src_dir, "images")):
        shutil.move(os.path.join(src_dir, "images", file_name), os.path.join(dest_dir, "images", file_name))
    for file_name in os.listdir(os.path.join(src_dir, "labels")):
        shutil.move(os.path.join(src_dir, "labels", file_name), os.path.join(dest_dir, "labels", file_name))

move_back_files(val_dir, train_dir)
move_back_files(test_dir, train_dir)

print("Recovered!")


Recovered!


## Clearml

In [13]:
from clearml import Task

#Clear ML Initialization
cl_task = Task.init(project_name=project,task_name=name)
logger = cl_task.get_logger()

UsageError: Current task already created and requested task name 'Mixed_YOLO_d1+d2+s1_test_2' does not match current task name 'Mixed_YOLO_d1+d2+s1_test_1'. If you wish to create additional tasks use `Task.create`, or close the current task with `task.close()` before calling `Task.init(...)`

In [ ]:
cl_task.close()

# 2 Training

In [10]:
from ultralytics import YOLO

model = YOLO(model)

results = model.train(project=project, name=name, exist_ok=exist_ok, data=data, imgsz=imgsz, task=task, device=device, epochs=epochs, batch=batch, lr0=lr0, cos_lr=cos_lr, weight_decay=weight_decay, dropout=dropout, optimizer=optimizer, momentum=momentum, seed=seed, plots=plots)

cl_task.close()

New https://pypi.org/project/ultralytics/8.2.60 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.41 🚀 Python-3.12.2 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/mnt/data/backup_pan/Dataset/Mixed_datasets/Det-dominik_v2-sf_v1.v2i.yolov8/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=Uni-Wood, name=Mixed_YOLO_d2+sf1_test_1, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=Fal

train: Scanning /mnt/data/backup_pan/Dataset/Mixed_datasets/Det-dominik_v2-sf_v1.v2i.yolov8/train/labels... 2178 images, 11 backgrounds, 0 corrupt: 100%|██████████| 2178/2178 [00:02<00:00, 937.59it/s]


train: New cache created: /mnt/data/backup_pan/Dataset/Mixed_datasets/Det-dominik_v2-sf_v1.v2i.yolov8/train/labels.cache


val: Scanning /mnt/data/backup_pan/Dataset/Mixed_datasets/Det-dominik_v2-sf_v1.v2i.yolov8/valid/labels... 256 images, 1 backgrounds, 0 corrupt: 100%|██████████| 256/256 [00:00<00:00, 1203.68it/s]

val: New cache created: /mnt/data/backup_pan/Dataset/Mixed_datasets/Det-dominik_v2-sf_v1.v2i.yolov8/valid/labels.cache


...

    200/200       6.7G     0.4675     0.2604     0.9402          2        640: 100%|██████████| 137/137 [00:25<00:00,  5.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  6.02it/s]

                   all        256        349      0.982       0.96      0.973      0.844



200 epochs completed in 1.684 hours.
Optimizer stripped from Uni-Wood/Mixed_YOLO_d2+sf1_test_1/weights/last.pt, 52.1MB
Optimizer stripped from Uni-Wood/Mixed_YOLO_d2+sf1_test_1/weights/best.pt, 52.1MB

Validating Uni-Wood/Mixed_YOLO_d2+sf1_test_1/weights/best.pt...
Ultralytics YOLOv8.2.41 🚀 Python-3.12.2 torch-2.3.1+cu121 CUDA:0 (NVIDIA RTX A5000, 24241MiB)
YOLOv8m summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.48it/s]


                   all        256        349       0.98      0.957      0.973      0.844
             wood_knot        255        349       0.98      0.957      0.973      0.844
Speed: 0.1ms preprocess, 3.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to Uni-Wood/Mixed_YOLO_d2+sf1_test_1


# 3 testing

## 3.1 path

In [20]:
from ultralytics import YOLO
import cv2
import os
import json

model_path = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Detection/YOLO/202407_Mixed_tests/Uni-Wood/Mixed_YOLO_d1+d2+s1_test_2/weights/best.pt"

image_folder = "/Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW"

save_path = "/Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/test_yolo_d1d2s1"

save_none_path = "/Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/test_yolo_d1d2s1/test_none"



## 3.2 resize

In [ ]:
# Resize images
resized_image_folder = os.path.join(image_folder, "resized")
os.makedirs(resized_image_folder, exist_ok=True)

for file in os.listdir(image_folder):
    if file.endswith(('.jpg', '.JPG', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (640, 640))
        resized_img_path = os.path.join(resized_image_folder, os.path.splitext(file)[0] + "_resize" + os.path.splitext(file)[1])
        cv2.imwrite(resized_img_path, img_resized)
image_folder = resized_image_folder


## 3.3 test

In [21]:
model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    results = model(img_path)
    img = cv2.imread(img_path)

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            # box contains prediction information
            confidence = box.conf[0]
            if confidence > 0.4:
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 5)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 5)

    unique_save_path = os.path.join(save_path, os.path.basename(img_path))
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")



image 1/1 /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/M1_0001.jpeg: 640x480 (no detections), 242.9ms
Speed: 2.3ms preprocess, 242.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 480)
Image saved: /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/test_yolo_d1d2s1/M1_0001.jpeg

WARNING ⚠️ Image Read Error /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/._M1_0001.jpeg


ValueError: need at least one array to stack

In [18]:
model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

if not os.path.exists(save_none_path):
    os.makedirs(save_none_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    results = model(img_path)
    img = cv2.imread(img_path)

    original_filename = os.path.basename(img_path)
    new_filename = "detected_" + original_filename

    image_results = {
        "image": new_filename,
        "boxes": []
    }

    detected_any = False

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            # box contains prediction information
            confidence = float(box.conf[0])
            if confidence > 0.1:
                detected_any = True
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 5)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 5)

                image_results["boxes"].append({
                    "confidence": confidence,
                    "x1": int(xyxy[0]),
                    "y1": int(xyxy[1]),
                    "x2": int(xyxy[2]),
                    "y2": int(xyxy[3])
                })

    if detected_any:
        unique_save_path = os.path.join(save_path, new_filename)
        json_save_path = os.path.join(save_path, new_filename.replace(os.path.splitext(new_filename)[1], ".json"))
    else:
        unique_save_path = os.path.join(save_none_path, new_filename)
        json_save_path = os.path.join(save_none_path, new_filename.replace(os.path.splitext(new_filename)[1], ".json"))

    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")

    with open(json_save_path, 'w') as json_file:
        json.dump(image_results, json_file, indent=4)
        print(f"Results saved to: {json_save_path}")


image 1/1 /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/M1_0001.jpeg: 640x480 (no detections), 230.8ms
Speed: 2.0ms preprocess, 230.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)
Image saved: /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/test_yolo_d1d2s1/test_none/detected_M1_0001.jpeg
Results saved to: /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/test_yolo_d1d2s1/test_none/detected_M1_0001.json

WARNING ⚠️ Image Read Error /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/._M1_0001.jpeg


ValueError: need at least one array to stack

In [23]:
model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

if not os.path.exists(save_none_path):
    os.makedirs(save_none_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    try:
        results = model(img_path)
    except Exception as e:
        print(f"Model inference error on image {img_path}: {e}")
        continue

    img = cv2.imread(img_path)
    if img is None:
        print(f"Image Read Error: {img_path}")
        continue

    original_filename = os.path.basename(img_path)
    new_filename = "detected_" + original_filename

    image_results = {
        "image": new_filename,
        "boxes": []
    }

    detected_any = False

    for result in results:
        if not result.boxes:
            continue
        
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            # box contains prediction information
            confidence = float(box.conf[0])
            if confidence > 0.4:
                detected_any = True
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 5)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 5)

                image_results["boxes"].append({
                    "confidence": confidence,
                    "x1": int(xyxy[0]),
                    "y1": int(xyxy[1]),
                    "x2": int(xyxy[2]),
                    "y2": int(xyxy[3])
                })

    if detected_any:
        unique_save_path = os.path.join(save_path, new_filename)
        json_save_path = os.path.join(save_path, new_filename.replace(os.path.splitext(new_filename)[1], ".json"))
    else:
        unique_save_path = os.path.join(save_none_path, new_filename)
        json_save_path = os.path.join(save_none_path, new_filename.replace(os.path.splitext(new_filename)[1], ".json"))
        image_results["message"] = "No detections"

    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")

    with open(json_save_path, 'w') as json_file:
        json.dump(image_results, json_file, indent=4)
        print(f"Results saved to: {json_save_path}")


image 1/1 /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/M1_0001.jpeg: 640x480 (no detections), 242.8ms
Speed: 2.7ms preprocess, 242.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)
Image saved: /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/test_yolo_d1d2s1/test_none/detected_M1_0001.jpeg
Results saved to: /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/test_yolo_d1d2s1/test_none/detected_M1_0001.json

WARNING ⚠️ Image Read Error /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/._M1_0001.jpeg
Model inference error on image /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/._M1_0001.jpeg: need at least one array to stack

image 1/1 /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_Dominikaner/RAW/M1_0002.jpeg: 640x480 1 wood_knot, 248.0ms
Speed: 2.7ms preprocess, 248.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 480)
Image saved: /Volumes/Extreme SSD/Datasets_ssd/4_Maria_0710_D